In [ ]:

不需要通过DBMS，可直接访问数据，并且将整个数据库保存在单个文件中


In [ ]:
sql语句
CREATE TABLE TableName(ColumnName Type, ColumnName Type, ….)
INSERT INTO TableName VALUES(Value1, Value2, …..)
SELECT ColumnName, ColumnName,.... FROM Table
DROP TABLE Tablename


In [ ]:
sqlite数据类型
NULL         
INTEGER
REAL              float8字节浮点数
TEXT              字符串，默认Unicode,字符串前缀u，但是python本身默认为ASCII,
BLOB              buffer二进制数据Binary Large Object 图片，音频，或者其他字节序列


In [ ]:
连接
import sqlite3 as dbapi
con=dbapi.connect(‘population.db’)       参数为路径，若不存在则新建
cur=con.cursor()


In [ ]:
创建
cur.execute('CREATE TABLE PopByRegion(Region TEXT, Population INTEGER)')


In [ ]:
插入
cur.execute('INSERT INTO PopByRegion VALUES("Central Africa", 330993)')
cur.execute('INSERT INTO PopByRegion VALUES("Southeastern Africa", 743112)')
cur.execute('INSERT INTO PopByRegion VALUES("Japan", 100562)')


In [ ]:
保存
con.commit()


In [ ]:
查询
cur.execute('SELECT Region, Population FROM PopByRegion')
cur.execute(‘SELECT Region, Population FROM PopByRegion ORDER BY Region’)
cur.execute(‘SELECT Region, Population FROM PopByRegion ORDER BY Region ASC’)
ASC升序，DESC降序
cur.execute(‘SELECT * FROM PopByRegion’)
cur.execute(‘SELECT Region FROM PopByRegion WHERE Population > 1000’)
cur.execute(‘SELECT * FROM PopByRegion WHERE Region= “Japan”’)
cur.execute(‘SELECT Region FROM PopByRegion 
                    WHERE Population > 1000 AND Region < “L”’)    还可使用OR, NOT


In [ ]:


更新
cur.execute(‘UPDATE PopByRegion SET  Population=10060 WHERE Region=”Japan”’)


In [ ]:
删除
cur.execute(‘DELETE FROM PopByRegion WHERE Population < “L”’)
cur.execute(‘DROP TABLE PopByRegion’)
DROP要求目标表没有外部依赖，所以在执行DROP之前要先修改表与表之间的外键约束



In [ ]:


print cur.fetchone()     以元组的形式返回每条记录，为空时返回None，默认Unicode
con.text_factory=str
print cur.fetchone()     返回str类型字符串
print cur.fetchall()       返回查询产生的全部数据


In [ ]:
事务transaction
对数据库操作的一个封装，撤销操作时需要回滚roll back


In [ ]:
借书
cur.execute('SELECT SignedOut FROM Books WHERE ISBN = "%s"' % isbn)
signedOut = cur.fetchone()[0]
cur.execute('UPDATE Books SET SignedOut = %d
             WHERE ISBN = "%s"' % (signedOut + 1, isbn))
cur.commit()


In [ ]:

还书
cur.execute('SELECT SignedOut FROM Books WHERE ISBN = "%s"' % isbn)
signedOut = cur.fetchone()[0]
cur.execute('UPDATE Books SET SignedOut = %d
             WHERE ISBN = "%s"' % (signedOut - 1, isbn))
cur.commit()



In [ ]:
缺失值
cur.execute(‘CREATE TABLE Test (Region TEXT NOT NULL, Population INTEGER)’)
有时用0，空字符串，false要比NULL更合适，当你对数据一无所知时，应该只用NULL
表的连接join
cur.execute('CREATE TABLE PopByCountry(Region TEXT, Country TEXT,
                 Population INTEGER)')
                          
countries = [("Eastern Asia", "DPR Korea", 24056), ("Eastern Asia",
"Hong Kong (China)", 8764), ("Eastern Asia", "Mongolia", 3407), ("Eastern
Asia", "Republic of Korea", 41491), ("Eastern Asia", "Taiwan", 1433),
("North America", "Bahamas", 368), ("North America", "Canada", 40876),
("North America", "Greenland", 43), ("North America", "Mexico", 126875),
("North America", "United States", 493038)]


In [ ]:
循环插入
for c in countries:
    cur.execute('INSERT INTO PopByCountry VALUES (?, ?, ?)', (c[0], c[1], c[2]))
con.commit()



In [ ]:
内连接
INNER JOIN
cross product
cur.execute('''SELECT PopByRegion.Region, PopByCountry.Country
                     FROM   PopByRegion INNER JOIN PopByCountry
                     WHERE  (PopByRegion.Region = PopByCountry.Region)
                     AND    (PopByRegion.Population > 1000000)''')
print cur.fetchall()

cur.execute('''SELECT PopByRegion.Region
                     FROM PopByRegion INNER JOIN PopByCountry
                     WHERE (PopByRegion.Region = PopByCountry.Region)
                     AND ((PopByCountry.Population * 1.0) / PopByRegion.Population > 0.10)''')
print cur.fetchall()


In [ ]:
去除重复项
cur.execute('''SELECT DISTINCT PopByRegion.Region
                     FROM PopByRegion INNER JOIN PopByCountry
                     WHERE (PopByRegion.Region = PopByCountry.Region)
                     AND ((PopByCountry.Population * 1.0) / PopByRegion.Population > 0.10)''')
print cur.fetchall()


In [ ]:

键和约束
键值是唯一的

主键primary key
cur.execute('CREATE TABLE PopByRegion (Region TEXT NOT NULL,
                   Population INTEGER NOT NULL, PRIMARY KEY (Region))');
多个主键
cur.execute('''CREATE TABLE PopByCountry(Region TEXT NOT NULL,
                    Country TEXT NOT NULL,
                    Population INTEGER NOT NULL,
                    CONSTRAINT Country_Key PRIMARY KEY (Region, Country))''')

外键foreign key



In [ ]:

聚合
cur.execute('SELECT SUM (Population) FROM PopByRegion')

聚合函数
AVG
MIN
MAX
COUNT   非空值的数量
SUM       
ABS


In [ ]:

分组
cur.execute('SELECT SUM (Population) FROM PopByCountry GROUP BY Region')



In [ ]:
自连接
对一个表中不同行的值进行比较
cur.execute('''SELECT A.Country, B.Country
                     FROM   PopByCountry A INNER JOIN PopByCountry B
                     WHERE  (ABS(A.Population - B.Population) <= 1000)
                     AND    (A.Country != B.Country)''')
加上 A.Country< B.Country可以去掉重复


In [ ]:
嵌套查询
nested query
cur.execute('''SELECT DISTINCT Region
                     FROM                    PopByCountry
                     WHERE                  (PopByCountry.Population != 8764)''')

In [ ]:
无法获得正确结果

cur.execute('''SELECT DISTINCT Region
                     FROM                    PopByCountry
                     WHERE                  Region NOT IN
                                   (SELECT DISTINCT Region
                                     FROM                    PopByCountry
                                     WHERE                  (PopByCountry.Population = 8764))''')
嵌套查询常常被用于处理含有否定推断的问题
